# Lekce 2

## Kontingenční tabulky

Pojem `pivot` obvykle překládáme jako kontingenční tabulka. Jde o talku, která porovnává dvě různé tzv. kategorie - jedna kategorie je použitá pro sloupce a druhá pro řádky. Pomocí kontingenční tabulky můžeme například porovnat tržby obchodních manažerů v jednotlivých zemích.

Pro sestavení kontingenční tabulky použijeme metodu `pivot_table`. Potřebujeme nastavit následující parametry:
- Sloupce, které použijeme pro sloupcovou a řádkovou dimenzi.
- Sloupec, který využijeme pro hodnoty, a agregační funkci.

Pokud chceme porovnat tržby sales manažerů pro jednotlivé země, použijeme sloupce `sales_manager` a `country` pro sloupce a řádky. Do "vnitřku" tabulky vložíme hodnoty sloupce `contract_value` a protože chceme vidět celkovou hodnotu, jako funkci použijeme `numpy.sum`.

Abychom si výsledky výpočtů trochu zpřehlednili, můžeme vydělit hodnotu sloupce `contract_value` milionem.

In [1]:
import pandas

df_actual = pandas.read_csv("data/sales_actual.csv")
df_actual.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/sales_actual.csv'

In [ ]:
df_actual["contract_value"] = df_actual["contract_value"] / 1_000_000

Zkusme nejprve vytvořit přehledovou tabulku pomocí standardní agregace. Provedeme agregaci podle sloupců `sales_manager` a `country` a použijeme agregační funkci `sum()` pro sloupec `contract_value`.

In [ ]:
df_actual_aggregated = df_actual.groupby(["sales_manager", "country"])["contract_value"].sum()
df_actual_aggregated

Nyní si zkusíme zobrazit stejnou informaci pomocí kontingenční tabulky, použijeme tedy funkci `pivot_table`.

Pozor, nepleť si funkci `pivot_table` a `pivot`! Funkci `pivot` též vytváří kontingenční tabulku, provádí však jen polovinu toho, co `pivot_table`. Přeskládá hodnoty v tabulce, ale neprovádí agregaci. Lze ji tedy použít pouze v případě, že pro každou kombinaci hodnot sloupců máš maximálně jeden záznam.

In [ ]:
df_actual_pivot = pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=sum, margins=True)
df_actual_pivot

Často je zajímavé zobrazit si i celkový součet pro řádky a sloupce, ten získáme pomocí parametru `margins`.

In [ ]:
df_actual_pivot_count = pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=sum, margins=True)
df_actual_pivot_count

Zkusme nyní změnit agregační funkci a uvažujme, že nás zajímá počet uzavřených kontraktů pro jednotlivé země
a sales manažery.

In [ ]:
pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=len, margins=True)

## Dobrovolné čtení na doma

### Teplotní mapa

I kontingenční tabulku můžeme prezentovat jako graf, hodí se například graf typu teplotní mapa (Heat Map). Ten vypadá podobně jako kontingenční tabulka, hodnoty jsou ale podbarveny barevnou škálou. Vygenerování grafu modulem `matplotlib` je poměrně náročné, proto je v tomto případně lepší použít "nadstavbu", kterou je modul `seaborn`.

In [ ]:
import seaborn as sns
sns.heatmap(df_actual_pivot, annot=True, fmt=".1f", linewidths=.5, cmap="YlGn")

### Relativní hodnoty v kontingenční tabulce

Občas mohou být zajímavější relativní hodnoty než absolutní. Uvažujme například situaci, kdy chceme vidět, kolik procent celkových tržeb v jednotlivých zemích zajistili jednotliví obchodníci. Abychom tuto informaci získali, vydělíme tržby jednotlivých obchodníků v jednotlivých zemích tržbami v celé zemi. Vydělení provedeme pomocí metody `div`.

In [ ]:
df_actual_pivot_percentage = df_actual_pivot.div( df_actual_pivot.iloc[:,-1], axis=0)
df_actual_pivot_percentage

V tabulce například vidíme, že 96 % tržeb v Kanadě zajistil Jones Leon, 49 % tržeb ve Francii Thomas Denise atd.

Můžeme se na data podívat i z opačného úhlu a podívat se na procentuální rozdělení tržeb obchodníků v rámci zemí.
K tomu potřebujeme vydělit tržby obchodníků v jednotlivých zemí celkovými tržbami jednotlivých obchodníků.

In [ ]:
df_actual_pivot_percentage = df_actual_pivot.div( df_actual_pivot.iloc[-1,:], axis=1)
df_actual_pivot_percentage

Z této tabulky vidíme, že Jones Leon zajistil třetinu svých tržeb v Kanadě a další třetinu ve Francii, Morgan Sara
polovinu tržeb ve Spojeném království atd.